# Ejemplo de `fastText` con `gensim`


En este cuaderno replicamos [las operaciones con los vectores de Word2Vec](word2vec.ipynb) pero utilizando los [vectores de palabras `fastText`](https://fasttext.cc/docs/en/crawl-vectors.html), entrenados por Facebook AI utilizando [Common Crawl](https://commoncrawl.org/) y Wikipedia. 

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Lo primero, es asegurarnos de que hemos descargado los vectores preentrenados. Si no lo has hecho antes, descarga y descomprime [la versión en texto plan con extensión `vec` de los vectores en español](https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz).

In [ ]:
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.vec.gz
#!gunzip cc.es.300.vec.gz

Cargamos los modelos usando `gensim`. Paciencia, esta operación tarda varios minutos:

In [4]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("/home/victor/data/fasttext/cc.es.300.vec")

2019-07-21 19:39:27,755 : INFO : loading projection weights from /home/victor/data/fasttext/cc.es.300.vec
2019-07-21 19:39:27,760 : WARNING : this function is deprecated, use smart_open.open instead
2019-07-21 19:48:40,637 : INFO : loaded (2000000, 300) matrix from /home/victor/data/fasttext/cc.es.300.vec


## Probando nuestro modelo

El objeto `model` contiene una enorme matriz de números: una tabla, donde cada fila es uno de los términos del vocabulario reconocido y cada columna es una de las características que permiten modelar el significado de dicho término.

Cada término del vocabulario está representado como un vector con 300 dimensiones. Como en `word2vec`, estos vectors son densos.

In [6]:
print(model["azul"])
print(model["verde"])
print(model["microsoft"])

[ 6.500e-03 -6.800e-03  7.120e-02 -1.757e-01  2.420e-02 -2.310e-02
  4.140e-02  5.100e-03  2.760e-02 -6.410e-02  1.608e-01 -7.210e-02
  9.870e-02  1.394e-01 -1.339e-01  4.430e-02 -1.440e-02 -4.600e-02
  1.570e-02  4.790e-02  2.300e-03  5.350e-02 -4.360e-02  7.870e-02
 -2.040e-02 -3.250e-02 -3.510e-02 -1.228e-01 -7.700e-02  6.260e-02
 -7.540e-02 -2.500e-03  2.540e-02  4.900e-02  5.500e-03 -4.200e-02
 -8.050e-02 -4.510e-02 -1.040e-01  4.830e-02  2.030e-02  8.590e-02
 -9.100e-03 -1.151e-01 -3.930e-02  5.640e-02  8.470e-02  1.091e-01
 -4.950e-02  2.000e-02 -6.350e-02  5.320e-02  4.250e-02  8.600e-03
  6.660e-02  1.803e-01 -1.090e-02  9.990e-02  1.375e-01  6.540e-02
 -8.810e-02  4.300e-03  2.520e-02 -4.000e-04  1.213e-01 -6.600e-02
 -1.602e-01  1.213e-01  3.050e-02 -1.188e-01 -1.636e-01  7.570e-02
  5.680e-02  4.920e-02 -9.900e-03  3.750e-02  2.680e-02 -1.840e-02
  5.950e-02  6.000e-03  3.940e-02  9.880e-02  6.470e-02 -4.060e-02
  4.250e-02  4.360e-02  3.710e-02 -1.070e-01 -8.830e-02 -1.477

Estos vectores no nos dicen mucho, salvo que contienen números muy pequeños :-/

El mismo objeto `model` permite acceder a una serie de funcionalidades ya implementadas que nos van a permitir evaluar formal e informalmente el modelo. Por el momento, nos contentamos con los segundo: vamos a revisar visualmente los significados que nuestro modelo ha aprendido por su cuenta. 

Podemos calcular la similitud semántica entre dos términos usando el método `similarity`, que nos devuelve un número entre 0 y 1:

In [9]:
print("hombre - mujer", model.similarity("hombre", "mujer"))

print("perro - gato", model.similarity("perro", "gato"))

print("gato - periódico", model.similarity("gato", "periódico"))

print("febrero - azul", model.similarity("febrero", "azul"))

hombre - mujer 0.6847727
perro - gato 0.81946486
gato - periódico 0.23872271
febrero - azul 0.0820393


Podemos seleccionar el término que no encaja a partir de una determinada lista de términos usando el método `doesnt_match`:

In [12]:
lista1 = "madrid barcelona gonzález washington".split()
print("en la lista {} sobra: {}".format(lista1, model.doesnt_match(lista1)))

lista2 = "psoe pp ciu ronaldo".split()
print("en la lista {} sobra: {}".format(lista2, model.doesnt_match(lista2)))

lista3 = "publicaron declararon soy negaron".split()
print("en la lista {} sobra: {}".format(lista3, model.doesnt_match(lista3)))

lista4 = "homero saturno cervantes shakespeare cela".split()
print("en la lista {} sobra: {}".format(lista4, model.doesnt_match(lista4)))

lista5 = "madrid barcelona alpedrete marsella".split()
print("en la lista {} sobra: {}".format(lista5, model.doesnt_match(lista5)))

2019-07-21 19:56:01,740 : INFO : precomputing L2-norms of word weight vectors


en la lista ['madrid', 'barcelona', 'gonzález', 'washington'] sobra: gonzález
en la lista ['psoe', 'pp', 'ciu', 'ronaldo'] sobra: ronaldo
en la lista ['publicaron', 'declararon', 'soy', 'negaron'] sobra: soy
en la lista ['homero', 'saturno', 'cervantes', 'shakespeare', 'cela'] sobra: cela
en la lista ['madrid', 'barcelona', 'alpedrete', 'marsella'] sobra: alpedrete


/home/victor/.virtualenvs/ds/lib/python3.7/site-packages/gensim/models/keyedvectors.py:876: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


Podemos buscar los términos más similares usando el método `most_similar` de nuestro modelo:

In [14]:
terminos = "psoe chicago rajoy enero amarillo microsoft iberia ronaldo messi atlético 2019".split()

for t in terminos:
    print("{} ==> {}".format(t, model.most_similar(t)))

psoe ==> [('ppsoe', 0.8005370497703552), ('Psoe', 0.7764014005661011), ('pesoe', 0.7492713332176208), ('rajoy', 0.7394102811813354), ('pnv', 0.6976327896118164), ('aznar', 0.6966044902801514), ('PsoE', 0.6826643943786621), ('PSOE', 0.6822848916053772), ('PSOE.', 0.6781550645828247), ('PSOE.Y', 0.675744354724884)]
chicago ==> [('Chicago', 0.6816766262054443), ('Chicago.', 0.6009481549263), ('brooklyn', 0.5926045775413513), ('oakland', 0.5809924602508545), ('seattle', 0.5727728605270386), ('houston', 0.5660635232925415), ('minnesota', 0.5595453977584839), ('manhattan', 0.5574262142181396), ('baltimore', 0.5437635183334351), ('detroit', 0.5420485138893127)]
rajoy ==> [('rubalcaba', 0.7588464617729187), ('aznar', 0.7570807337760925), ('psoe', 0.7394102811813354), ('ppsoe', 0.6995846033096313), ('cospedal', 0.6919615268707275), ('merkel', 0.6574246883392334), ('solbes', 0.6568419933319092), ('pnv', 0.6447515487670898), ('Rajoy', 0.6433820128440857), ('barcenas', 0.6415294408798218)]
enero =

Con el mismo método `most_similar` podemos combinar vectores de palabras tratando de jugar con los rasgos semánticos de cada una de ellas para descubrir nuevas relaciones.

In [16]:
print("mujer que ejerce la autoridad en una alcaldía ==> alcalde + mujer - hombre")
most_similar = model.most_similar(positive=["alcalde", "mujer"], negative=["hombre"], topn=3)
for item in most_similar:
    print(item)

print("monarca soberano ==> reina + hombre - mujer")    
most_similar = model.most_similar(positive=["reina", "hombre"], negative=["mujer"], topn=3)
for item in most_similar:
    print(item)
    
print("capital de Alemania ==> moscú + alemania - rusia")
most_similar = model.most_similar(positive=["moscú", "alemania"], negative=["rusia"], topn=3)
for item in most_similar:
    print(item)

print("presidente de Francia ==> obama + francia - eeuu")
most_similar = model.most_similar(positive=["obama", "francia"], negative=["eeuu"], topn=3)
for item in most_similar:
    print(item)

mujer que ejerce la autoridad en una alcaldía ==> alcalde + mujer - hombre
('alcaldesa', 0.8514840602874756)
('Alcaldesa', 0.7635220289230347)
('alcadesa', 0.7297311425209045)
monarca soberano ==> reina + hombre - mujer
('rey', 0.6823060512542725)
('monarca', 0.6184303760528564)
('reino', 0.6015368700027466)
capital de Alemania ==> moscú + alemania - rusia
('munich', 0.5816982984542847)
('austria', 0.5530569553375244)
('múnich', 0.5446099042892456)
presidente de Francia ==> obama + francia - eeuu
('sarkozy', 0.511267364025116)
('henri', 0.4758860766887665)
('paris', 0.47072291374206543)
